NAMA : TIO DWI SATRIO
NPM  : 5210411287
Machine	Learning - Programming Assignment 4

In [56]:
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB

class NaiveBayesClassifier:
    def __init__(self):
        self.groups = []
        self.group_probabilities = {}
        self.feature_probabilities = {}

    def train(self, X, y):
        # Jumlah data train pada tiap group
        group_counts = {}
        for label in y:
            if label not in group_counts:
                group_counts[label] = 0
            group_counts[label] += 1

        # Probabilitas group
        samples_total = len(y)
        self.groups = list(set(y))
        for label in self.groups:
            self.group_probabilities[label] = group_counts[label] / samples_total

        # Probabilitas fitur pada tiap group
        self.feature_probabilities = {}
        for label in self.groups:
            self.feature_probabilities[label] = {}

            # Sampel & Label yang SAMA, akan digabungkan
            samples = [X[i] for i in range(samples_total) if y[i] == label]

            # Probabilitas fitur untuk tiap atribut
            for i in range(len(X[0])):
                feature_counts = {}
                for sample in samples:
                    feature_value = sample[i]
                    if feature_value not in feature_counts:
                        feature_counts[feature_value] = 0
                    feature_counts[feature_value] += 1

                # Menghitung probabilitas fitur berdasarkan jumlah kemunculannya
                self.feature_probabilities[label][i] = {}
                for feature_value in feature_counts:
                    self.feature_probabilities[label][i][feature_value] = (
                        feature_counts[feature_value] / group_counts[label]
                    )

    def predict(self, X):
        predictions = []
        for sample in X:
            predicted_group = None
            max_probability = -1

            # Mencari kelas dengan probabilitas terbesar
            for label in self.groups:
                probability = self.group_probabilities[label]

                for i in range(len(sample)):
                    feature_value = sample[i]
                    if feature_value in self.feature_probabilities[label][i]:
                        probability *= self.feature_probabilities[label][i][feature_value]

                if probability > max_probability:
                    max_probability = probability
                    predicted_group = label

            predictions.append(predicted_group)

        return predictions

# Library untuk mendapatkan angka secara acak
import random

# Data pelatihan
X_train = []
y_train = []

# Membuat data secara random, sebanyak 500 data
for _ in range(500):
    # Menghasilkan tinggi badan antara 160-190 cm secara acak
    tinggi_badan = random.randint(160, 190)

    # Menghasilkan berat badan antara 60-90 kg secara acak
    berat_badan = random.randint(60, 120)

    # Menghasilkan jenis rambut secara acak: jamet, ikal, lurus, keriting, dan botak.
    jenis_rambut = random.choice(['jamet', 'ikal', 'lurus', 'keriting', 'botak'])

    X_train.append([tinggi_badan, berat_badan, jenis_rambut])

    # Menghasilkan label "pelaku" atau "bukan pelaku" secara acak
    label = random.choice(['pelaku', 'bukan pelaku'])
    y_train.append(label)

# Data pengujian (Ciri-ciri pelaku)
X_test = [
    [177, 72, 'jamet']
]

X_train

# Menggabungkan data pelatihan dan pengujian untuk encoding
all_data = X_train + X_test

# Menginisialisasi dan fit LabelEncoder dengan data gabungan
label_encoder = LabelEncoder()
label_encoder.fit([data[2] for data in all_data])  # Mengambil jenis_rambut dari semua data

# Convert jenis_rambut menjadi angka menggunakan LabelEncoder
X_train_encoded = [[x[0], x[1], label_encoder.transform([x[2]])[0]] for x in X_train]
X_test_encoded = [[x[0], x[1], label_encoder.transform([x[2]])[0]] for x in X_test]

# Membuat objek klasifikasi Gaussian Naive Bayes
naive_bayes_classifier = GaussianNB()

# Melatih klasifikasi Naive Bayes
naive_bayes_classifier.fit(X_train_encoded, y_train)

# Memprediksi kelas untuk data pengujian
predictions = naive_bayes_classifier.predict(X_test_encoded)

# Menampilkan hasil prediksi
print("Prediksi kelas:", predictions)

# Mendapatkan probabilitas prediksi
probabilities = naive_bayes_classifier.predict_proba(X_test_encoded)[0]

# Menghitung probabilitas dalam persentase
probabilities_percent = {}
for i, label in enumerate(naive_bayes_classifier.classes_):
    probabilities_percent[label] = probabilities[i] * 100

# Menampilkan hasil probabilitas
for label in probabilities_percent:
    print(f"Probabilitas {label}: {probabilities_percent[label]:.2f}%")

Prediksi kelas: ['pelaku']
Probabilitas bukan pelaku: 47.76%
Probabilitas pelaku: 52.24%


MultiLayer Perceptrons

In [57]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

import numpy as np

class MLPModel:
    def __init__(self):
        self.model = MLPClassifier(hidden_layer_sizes=(100, 100), activation='relu', random_state=42)

    def collect_training_data(self):
        # Data pelatihan yang telah dikumpulkan
        self.features = [
            {'sepatu': 42, 'tiket_bis': 1, 'jejak_tinta': 'URO', 'lokasi_persembunyian': 'pabrik'},
            {'sepatu': 41, 'tiket_bis': 0, 'jejak_tinta': 'ORU', 'lokasi_persembunyian': 'gudang'},
            {'sepatu': 43, 'tiket_bis': 0, 'jejak_tinta': 'URO', 'lokasi_persembunyian': 'apartemen'},
            {'sepatu': 42, 'tiket_bis': 1, 'jejak_tinta': 'URO', 'lokasi_persembunyian': 'pabrik'},
            {'sepatu': 44, 'tiket_bis': 1, 'jejak_tinta': 'OUR', 'lokasi_persembunyian': 'pabrik'},
            {'sepatu': 41, 'tiket_bis': 0, 'jejak_tinta': 'ROR', 'lokasi_persembunyian': 'pabrik'},
        ]
        
        self.labels = [data['lokasi_persembunyian'] for data in self.features]

        # Menambahkan data pelatihan tambahan dengan looping
        for _ in range(500):
            sepatu = np.random.randint(40, 46)
            tiket_bis = np.random.randint(2)
            jejak_tinta = np.random.choice(['URO', 'ROUR', 'OUR', 'ORU'])
            lokasi_persembunyian = np.random.choice(['pabrik', 'gudang', 'apartemen'])
            data = {'sepatu': sepatu, 'tiket_bis': tiket_bis, 'jejak_tinta': jejak_tinta, 'lokasi_persembunyian': lokasi_persembunyian}
            self.features.append(data)
            self.labels.append(lokasi_persembunyian)

    def display_training_data(self):
        for i, data in enumerate(self.features):
            print(f"Data pelatihan {i+1}:")
            print("Sepatu:", data['sepatu'])
            print("Tiket Bis:", data['tiket_bis'])
            print("Jejak Tinta:", data['jejak_tinta'])
            print("Lokasi Persembunyian:", data['lokasi_persembunyian'])
            print()

    def preprocess_data(self):
        # Mengubah data kategorikal menjadi numerik dengan LabelEncoder dan OneHotEncoder
        le = LabelEncoder()
        self.features_encoded = [{'sepatu': data['sepatu'], 'tiket_bis': data['tiket_bis'], 'jejak_tinta': le.fit_transform([data['jejak_tinta']])[0]} for data in self.features]

        ohe = OneHotEncoder(sparse=False)
        self.features_encoded = [{'sepatu': data['sepatu'], 'tiket_bis': data['tiket_bis'], 'jejak_tinta': ohe.fit_transform(np.array(data['jejak_tinta']).reshape(-1, 1)).tolist()[0]} for data in self.features_encoded]

        # Convert labels to 1D array
        self.labels = np.array(self.labels)

    def combine_features(self):
        # Menggabungkan semua fitur
        self.X = np.array([[data['sepatu'], data['tiket_bis']] + data['jejak_tinta'] for data in self.features_encoded])

    def train_model(self):
        # Melatih model Neural Network Multi-Layer Perceptron dengan data pelatihan yang telah dikumpulkan dan fitur-fitur yang telah digabungkan
        self.model.fit(self.X, self.labels)

    def predict_location(self, data):
      # Prediksi lokasi persembunyian berdasarkan data baru
      data_encoded = {'sepatu': data['sepatu'], 'tiket_bis': data['tiket_bis'], 'jejak_tinta': LabelEncoder().fit_transform([data['jejak_tinta']])[0]}
      data_encoded = {'sepatu': data_encoded['sepatu'], 'tiket_bis': data_encoded['tiket_bis'], 'jejak_tinta': OneHotEncoder(sparse=False).fit_transform(np.array(data_encoded['jejak_tinta']).reshape(-1, 1)).tolist()[0]}
      X = np.array([[data_encoded['sepatu'], data_encoded['tiket_bis']] + data_encoded['jejak_tinta']])
      return self.model.predict(X)

# Contoh penggunaan Neural Network Multilayer Pereptron
mlp_model = MLPModel()
mlp_model.collect_training_data()
mlp_model.display_training_data()
mlp_model.preprocess_data()
mlp_model.combine_features()
mlp_model.train_model()


# Data baru yang akan diprediksi lokasi persembunyiannya
new_data = {'sepatu': 42, 'tiket_bis': 1, 'jejak_tinta': 'URO'}
predicted_location = mlp_model.predict_location(new_data)

predicted_location = np.squeeze(predicted_location)  # Mengubah bentuk array menjadi 1D

print('Lokasi persembunyian yang diprediksi:', predicted_location)


Data pelatihan 1:
Sepatu: 42
Tiket Bis: 1
Jejak Tinta: URO
Lokasi Persembunyian: pabrik

Data pelatihan 2:
Sepatu: 41
Tiket Bis: 0
Jejak Tinta: ORU
Lokasi Persembunyian: gudang

Data pelatihan 3:
Sepatu: 43
Tiket Bis: 0
Jejak Tinta: URO
Lokasi Persembunyian: apartemen

Data pelatihan 4:
Sepatu: 42
Tiket Bis: 1
Jejak Tinta: URO
Lokasi Persembunyian: pabrik

Data pelatihan 5:
Sepatu: 44
Tiket Bis: 1
Jejak Tinta: OUR
Lokasi Persembunyian: pabrik

Data pelatihan 6:
Sepatu: 41
Tiket Bis: 0
Jejak Tinta: ROR
Lokasi Persembunyian: pabrik

Data pelatihan 7:
Sepatu: 44
Tiket Bis: 0
Jejak Tinta: ORU
Lokasi Persembunyian: gudang

Data pelatihan 8:
Sepatu: 40
Tiket Bis: 1
Jejak Tinta: ROUR
Lokasi Persembunyian: gudang

Data pelatihan 9:
Sepatu: 45
Tiket Bis: 1
Jejak Tinta: ROUR
Lokasi Persembunyian: apartemen

Data pelatihan 10:
Sepatu: 42
Tiket Bis: 1
Jejak Tinta: ORU
Lokasi Persembunyian: gudang

Data pelatihan 11:
Sepatu: 44
Tiket Bis: 0
Jejak Tinta: ORU
Lokasi Persembunyian: pabrik

Data pelati

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

Lokasi persembunyian yang diprediksi: pabrik


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
